In [14]:
import os

import tensorflow as tf
from tensorflow import keras

Usamos el conjunto de datos MNIST para la demostración de cómo guardar y cargar pesos.

In [15]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

Modelo.

In [16]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation = 'relu', input_shape = (784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(optimizer = 'adam',
                  loss = tf.losses.SparseCategoricalCrossentropy(from_logits = True),
                  metrics = [tf.metrics.SparseCategoricalAccuracy()])
    
    return model

model = create_model()

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 512)               401920    
                                                                 
 dropout_6 (Dropout)         (None, 512)               0         
                                                                 
 dense_13 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Guardamos puntos de control durante el entrenamiento.

Podemos usar un modelo entrenado sin tener que volver a entrenarlo o retomar el entrenamiento donde lo dejamos en caso de que se interrumpa el proceso de entrenamiento. La callback tf.keras.callbacks.ModelCheckpoint nos permite guardar continuamente el modelo durante y al final del entrenamiento.

Creamos una callback que guarde los pesos SOLO durante el entrenamiento:

In [17]:
checkpoint_path = 'training_1/cp.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Creamos una callback que guarde los pesos del modelo.
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only = True,
                                                 verbose = 1)

# Entrenamos el modelo con el callback.
model.fit(train_images,
          train_labels,
          epochs = 10,
          validation_data = (test_images, test_labels),
          callbacks = [cp_callback])

Epoch 1/10
28/32 [=========================>....] - ETA: 0s - loss: 1.2052 - sparse_categorical_accuracy: 0.6540
Epoch 1: saving model to training_1\cp.ckpt
32/32 [==============================] - 2s 22ms/step - loss: 1.1492 - sparse_categorical_accuracy: 0.6700 - val_loss: 0.7207 - val_sparse_categorical_accuracy: 0.7790
Epoch 2/10
29/32 [==========================>...] - ETA: 0s - loss: 0.4451 - sparse_categorical_accuracy: 0.8739
Epoch 2: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 15ms/step - loss: 0.4383 - sparse_categorical_accuracy: 0.8770 - val_loss: 0.5483 - val_sparse_categorical_accuracy: 0.8330
Epoch 3/10
31/32 [============================>.] - ETA: 0s - loss: 0.2977 - sparse_categorical_accuracy: 0.9224
Epoch 3: saving model to training_1\cp.ckpt
32/32 [==============================] - 0s 15ms/step - loss: 0.2976 - sparse_categorical_accuracy: 0.9220 - val_loss: 0.4997 - val_sparse_categorical_accuracy: 0.8430
Epoch 4/10
27/32 [=======

Esto crea una única colección de archivos de puntos de control de Tensorflow que se actualiza al final de cada época:

In [18]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

Siempre que dos modelos compartan la misma arquitectura, pueden compartir pesos entre ellos. Por lo tanto, cuando restauremos un modelo de solo pesos, creamos un modelo con la misma arquitectura que el modelo original y luego establecemos sus pesos.

Ahora vamos a reconstruir un modelo nuevo, no entrenado y lo evaluamos con el conjunto de prueba. Un modelo no entrenado se desempeñará en nivelos de probabilidad(~10 % de precisión):

Modelo básico:

In [19]:
model_2 = create_model()

# Evaluamos el modelo.
loss, acc = model_2.evaluate(test_images, test_labels, verbose = 2)
print('Modelo no entrenado, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 2.4001 - sparse_categorical_accuracy: 0.0520 - 362ms/epoch - 11ms/step
Modelo no entrenado, accuracy:  5.20%


Cargamos los pesos desde el checkpoint y volvemos a evaluar.

In [20]:
# Cargamos los pesos.
model_2.load_weights(checkpoint_path)

# Re-evaluamos el modelo.
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print('Modelo con pesos agregados, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4233 - sparse_categorical_accuracy: 0.8620 - 107ms/epoch - 3ms/step
Modelo con pesos agregados, accuracy: 86.20%


Las callbacks ofrecen varias opciones para proporcionar nombres únicos para los checkpoints y ajustar la frecuencia de los puntos de control.

Entrenamos un modelo y guardamos los checkpoints con nombres exclusivos una vez cada cinco épocas:

In [21]:
# Incluimos la epoca en el nombre del archivo.
checkpoint_path = 'training_2/cp-{epoch:04d}.ckpt'
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Creamos un callback que guarde los pesos del modelo cada 5 épocas.
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True,
    save_freq = 5 * batch_size
)

model_3 = create_model()

model_3.save_weights(checkpoint_path.format(epoch = 0))

model_3.fit(train_images,
            train_labels,
            epochs = 50,
            batch_size = batch_size,
            callbacks = [cp_callback],
            validation_data = (test_images, test_labels),
            verbose = 0)


Epoch 5: saving model to training_2\cp-0005.ckpt

Epoch 10: saving model to training_2\cp-0010.ckpt

Epoch 15: saving model to training_2\cp-0015.ckpt

Epoch 20: saving model to training_2\cp-0020.ckpt

Epoch 25: saving model to training_2\cp-0025.ckpt

Epoch 30: saving model to training_2\cp-0030.ckpt

Epoch 35: saving model to training_2\cp-0035.ckpt

Epoch 40: saving model to training_2\cp-0040.ckpt

Epoch 45: saving model to training_2\cp-0045.ckpt

Epoch 50: saving model to training_2\cp-0050.ckpt


Veamos los checkpoints y elegimos el último.

In [22]:
os.listdir(checkpoint_dir)

['checkpoint',
 'cp-0000.ckpt.data-00000-of-00001',
 'cp-0000.ckpt.index',
 'cp-0005.ckpt.data-00000-of-00001',
 'cp-0005.ckpt.index',
 'cp-0010.ckpt.data-00000-of-00001',
 'cp-0010.ckpt.index',
 'cp-0015.ckpt.data-00000-of-00001',
 'cp-0015.ckpt.index',
 'cp-0020.ckpt.data-00000-of-00001',
 'cp-0020.ckpt.index',
 'cp-0025.ckpt.data-00000-of-00001',
 'cp-0025.ckpt.index',
 'cp-0030.ckpt.data-00000-of-00001',
 'cp-0030.ckpt.index',
 'cp-0035.ckpt.data-00000-of-00001',
 'cp-0035.ckpt.index',
 'cp-0040.ckpt.data-00000-of-00001',
 'cp-0040.ckpt.index',
 'cp-0045.ckpt.data-00000-of-00001',
 'cp-0045.ckpt.index',
 'cp-0050.ckpt.data-00000-of-00001',
 'cp-0050.ckpt.index']

In [23]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2\\cp-0050.ckpt'

Para probar, reiniciamos el modelo y cargamos el último punto de control:

In [24]:
model_4 = create_model()

model_4.load_weights(latest)

loss, acc = model_4.evaluate(test_images, test_labels, verbose = 2)
print('Modelo con pesos agregados versión 2, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4727 - sparse_categorical_accuracy: 0.8760 - 403ms/epoch - 13ms/step
Modelo con pesos agregados versión 2, accuracy: 87.60%


El código anterior almacena los pesos en una colección de archivos con formato de checkpoints que contienen solo los pesos entrenados en un formato binario. Los checkpoints contienen:

* Uno o más fragmentos que contienen los pesos de su modelo.
* Un archivo de índice que indica qué pesos se almacenan en qué fragmento.

Si entrenamos un modelo en una sola máquina, tendremos un fragmento con el sufijo: .data-00000-of-00001

### Guardamos pesos manualmente.

Con el método Model.save_weights vamos a guardar los pesos manualmente. De forma predeterminada, tf.keras y save_weights en particular, usa el formato de checkpoint de Tensorflow con una extensión .ckpt.

In [25]:
model_4.save_weights('./checkpoints/my_checkpoint')

model_4 = create_model()

model_4.load_weights('./checkpoints/my_checkpoint')

loss, acc = model_4.evaluate(test_images, test_labels, verbose = 2)
print('Modelo con pesos agregados versión 3, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 0s - loss: 0.4727 - sparse_categorical_accuracy: 0.8760 - 341ms/epoch - 11ms/step
Modelo con pesos agregados versión 3, accuracy: 87.60%


### Guardar todo el modelo.

Llamamos a model.save para guardar la arquitectura, los pesos y la configuración de entrenamiento de un modelo en un solo archivo/carpeta. Esto permite exporta un modelo para que pueda usarse sin acceso al código original de Python. Dado que se recuper ael estadoo de optimizador, puede reanudar el entrenamiento exactamente desde lo dejamos.

Un modelo completo se puede guardar en dos formatos de archivo diferentes (SavedModel y HDF5). El formato de SavedModel de TensorFlow es el formato de archivo predeterminado en TF2.x. Sin embargo, los modelos se pueden guardar en formato HDF5.

Guardar un modelo completamente funcional es muy útil: podemos cargarlo en TensorFlow.js (SavedModel, HDF5) y luego entrenarlo y ejecutarlo en navegadores web, o convertirlo para ejecutarlo en dispositivos móviles usando TensorFlow Lite (SavedModel, HDF5).

Los objetos personalizados (p. ej., modelos o capas subclasificados) requieren una atención especial al guardarlos y cargarlos.

### Formato de modelo guardado.

El formato de modelo guardado es otra forma de serializar modelos. Los modelos guardados en este formato se pueden restaurar usando tf.keras.models.load_model y son compatibles con Tensorflow Serving.

In [26]:
model_5 = create_model()
model_5.fit(train_images, train_labels, epochs = 5)

!mkdir -p saved_model
model_5.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 10ms/step - loss: 1.1343 - sparse_categorical_accuracy: 0.6810
Epoch 2/5
32/32 [==============================] - 0s 10ms/step - loss: 0.4262 - sparse_categorical_accuracy: 0.8840
Epoch 3/5
32/32 [==============================] - 0s 10ms/step - loss: 0.2864 - sparse_categorical_accuracy: 0.9270
Epoch 4/5
32/32 [==============================] - 0s 11ms/step - loss: 0.2105 - sparse_categorical_accuracy: 0.9480
Epoch 5/5
32/32 [==============================] - 0s 11ms/step - loss: 0.1496 - sparse_categorical_accuracy: 0.9720


INFO:tensorflow:Assets written to: saved_model/my_model\assets


INFO:tensorflow:Assets written to: saved_model/my_model\assets


El formato del modelo es un directorio que contiene un binario protofub y un checkpoint de Tensorflow.

Cargamos un modelo nuevo desde el modelo guardado:

In [27]:
new_model = tf.keras.models.load_model('saved_model/my_model')

new_model.summary()

Model: "sequential_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 512)               401920    
                                                                 
 dropout_11 (Dropout)        (None, 512)               0         
                                                                 
 dense_23 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


El modelo restaurado se compila con los mismos argumentos que el modelo original. Probemos evaluar y predecir con el modelo cargado:

In [28]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose = 2)
print('Modelo con pesos agregados versión 4, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4403 - sparse_categorical_accuracy: 0.8620 - 470ms/epoch - 15ms/step
Modelo con pesos agregados versión 4, accuracy: 86.20%
32/32 [==============================] - 0s 4ms/step
(1000, 10)


### Formato HDF5

Keras proporciona un formato de guardado básico utilizando el estándar HDF5.

In [29]:
model_6 = create_model()
model_6.fit(train_images, train_labels, epochs = 5)

model_6.save('my_model.h5')

Epoch 1/5
32/32 [==============================] - 2s 15ms/step - loss: 1.1354 - sparse_categorical_accuracy: 0.6840
Epoch 2/5
32/32 [==============================] - 0s 13ms/step - loss: 0.4248 - sparse_categorical_accuracy: 0.8730
Epoch 3/5
32/32 [==============================] - 0s 13ms/step - loss: 0.2811 - sparse_categorical_accuracy: 0.9240
Epoch 4/5
32/32 [==============================] - 1s 16ms/step - loss: 0.2066 - sparse_categorical_accuracy: 0.9520
Epoch 5/5
32/32 [==============================] - 0s 12ms/step - loss: 0.1613 - sparse_categorical_accuracy: 0.9680


Ahora, creamos el modelo con ese archivo.

In [30]:
new_model_2 = tf.keras.models.load_model('my_model.h5')

new_model_2.summary()

Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_24 (Dense)            (None, 512)               401920    
                                                                 
 dropout_12 (Dropout)        (None, 512)               0         
                                                                 
 dense_25 (Dense)            (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [31]:
loss, acc = new_model_2.evaluate(test_images, test_labels, verbose = 2)
print('Modelo con pesos agregados versión 5, accuracy: {:5.2f}%'.format(100 * acc))

32/32 - 1s - loss: 0.4399 - sparse_categorical_accuracy: 0.8660 - 712ms/epoch - 22ms/step
Modelo con pesos agregados versión 5, accuracy: 86.60%


Keras guarda modelos al inspeccionar sus arquitecturas. Esta técnica guarda todo:

* Los valores de peso.
* La arquitectura del modelo.
* La configuración de entrenamiento del modelo (lo que pasa al método .compile()).
* El optimizador y su estado, si lo hay (esto le permite reiniciar el entrenamiento donde lo dejó).

Keras no puede guardar los optimizadores v1.x (de tf.compat.v1.train ) ya que no son compatibles con los puntos de control. Para los optimizadores v1.x, deberíamos volver a compilar el modelo después de cargarlo, perdiendo el estado del optimizador.

La diferencia clave entre HDF5 y SavedModel es que HDF5 usa configuraciones de objetos para guardar la arquitectura del modelo, mientras que SavedModel guarda el gráfico de ejecución. Por lo tanto, los modelos guardados pueden guardar objetos personalizados, como modelos subclasificados y capas personalizadas, sin necesidad del código original.

Para guardar objetos personalizados en HDF5, debemos hacer lo siguiente:

1. Definir un método get_config en su objeto y, opcionalmente, un método de from_config. 

* get_config(self) devuelve un diccionario serializable JSON de parámetros necesarios para recrear el objeto.  

* from_config(cls, config) usa la configuración devuelta de get_config para crear un nuevo objeto. De forma predeterminada, esta función utilizará la configuración como kwargs de inicialización (return cls(**config)).

2. Pasemos el objeto al argumento custom_objects al cargar el modelo. El argumento debe ser un diccionario que mapee el nombre de la clase de cadena a la clase de Python. Por ejemplo tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})